In [1]:
import sys
sys.path.append('../')
from libraries.dataManipulation import *
import time
import datetime
import pandas as pd

In [2]:
%%capture
data = getDay(20220201,'ehv')

In [3]:
def division_by_direction(data):
    data['time'] = (data['date_time_utc'] - pd.Timestamp("1970-01-01")) // pd.Timedelta('1s')
    directions = direction_label(data)
    stats = directions['direction'].value_counts(normalize=True)*100
    if stats['undefined'] > 2:
        print('WARNING: Results might be wrong, large number of undefined direction objects.')
    data['direction'] = 'unassigned'
    directions_dict = directions.set_index('tracked_object')['direction'].to_dict()
    data['direction'] = data['tracked_object'].map(directions_dict)
    data_off = data[data['direction']=='offboarding']
    data_on = data[data['direction']=='onboarding']
    data_off = data_off.drop('direction',axis=1)
    data_on = data_on.drop('direction',axis=1)
    return (data_on, data_off)

In [4]:
(data_on, data_off) = division_by_direction(data)

In [5]:
train_data = SociophysicsDataHandler()
train_data.fetch_prorail_train_information(station='ehv')
train_data = train_data.train_information

trying to fetch: /storage/surfsara/ProRail_USE_LL_data/ehv/ehv_train_information_pss.parquet
data fetched. Accessible as <this-object>.train_information


In [6]:
train_data = train_data.drop(columns=['date', 'station'])

In [7]:
def to_unix_timestamp(dt):
        return dt.timestamp()

train_data['arrival_unix'] = train_data['arrival_time'].apply(to_unix_timestamp)
train_data['departure_unix'] = train_data['departure_time'].apply(to_unix_timestamp)

In [8]:
train_data_no_nan = train_data.dropna()
#limit rows to 1 row per train (individual door information deleted)
train_data_filtered = train_data_no_nan.drop_duplicates('arrival_time')

In [9]:
unix_timestamp_day_start = data_on.iloc[0]['time']
unix_timestamp_day_end = data_on.iloc[-1]['time']
day_train_data = train_data_filtered[(train_data_filtered['arrival_unix'] > unix_timestamp_day_start) & (train_data_filtered['departure_unix'] < unix_timestamp_day_end)]
day_train_data = day_train_data.reset_index(drop=True)

In [10]:
day_train_data

,train_number,arrival_time,departure_time,door_no,door_x,door_y,arrival_unix,departure_unix
0,818,2022-02-01 06:36:02+01:00,2022-02-01 06:37:38+01:00,1.0,7175.790848,3870.126635,1.643694e+09,1.643694e+09
1,820,2022-02-01 07:01:56+01:00,2022-02-01 07:06:32+01:00,1.0,10579.775716,3824.435563,1.643695e+09,1.643696e+09
2,824,2022-02-01 08:04:57+01:00,2022-02-01 08:06:59+01:00,1.0,6442.193299,-8620.026418,1.643699e+09,1.643699e+09
3,826,2022-02-01 08:32:58+01:00,2022-02-01 08:36:05+01:00,1.0,10542.193299,3824.940025,1.643701e+09,1.643701e+09
4,828,2022-02-01 09:02:36+01:00,2022-02-01 09:06:07+01:00,1.0,10387.387858,3827.017951,1.643703e+09,1.643703e+09
5,830,2022-02-01 09:32:34+01:00,2022-02-01 09:36:04+01:00,1.0,6975.790848,3872.811199,1.643704e+09,1.643705e+09
6,832,2022-02-01 10:02:45+01:00,2022-02-01 10:06:02+01:00,1.0,11375.790848,3813.750796,1.643706e+09,1.643706e+09
7,834,2022-02-01 10:32:44+01:00,2022-02-01 10:35:59+01:00,1.0,10887.387858,3820.306541,1.643708e+09,1.643708e+09
8,836,2022-02-01 11:02:28+01:00,2022-02-01 11:06:22+01:00,1.0,7375.790848,3867.442072,1.643710e+09,1.643710e+09
9,838,2022-02-01 11:31:51+01:00,2022-02-01 11:36:19+01:00,1.0,15175.790848,3762.744085,1.643712e+09,1.643712e+09


In [1]:
day_train_data = day_train_data.assign(train_position=lambda x: np.where(x['door_y'] > 2500, 'top', 'bottom'))

NameError: name 'day_train_data' is not defined

In [ ]:
day_train_data = day_train_data.drop(columns=['arrival_time', 'departure_time', 'door_no', 'door_x', 'door_y'])

In [ ]:
trains_in_day = len(day_train_data.index)
data_on['train'] = 'no train' 
for train in range(trains_in_day):
    arrival = day_train_data.loc[train]['arrival_unix']
    departure = day_train_data.loc[train]['departure_unix']
    if day_train_data.loc[train]['train_position'] == 'top':
        data_on.loc[(data_on['time'] > arrival) & (data_on['time'] < departure) & (data_on['train'] == 'no train'), 'train'] = 'top'
        data_on.loc[(data_on['time'] > arrival) & (data_on['time'] < departure) & (data_on['train'] == 'bottom'), 'train'] = 'both'
    elif day_train_data.loc[train]['train_position'] == 'bottom':
        data_on.loc[(data_on['time'] > arrival) & (data_on['time'] < departure) & (data_on['train'] == 'no train'), 'train'] = 'bottom'
        data_on.loc[(data_on['time'] > arrival) & (data_on['time'] < departure) & (data_on['train'] == 'top'), 'train'] = 'both'

In [ ]:
data_on_train_top = data_on[data_on['train']=='top']
data_on_train_bottom = data_on[data_on['train']=='bottom']
data_on_train_both = data_on[data_on['train']=='both']
data_on_no_train = data_on[data_on['train']=='no train']

In [ ]:
data_on_train_top = data_on_train_top.drop('train',axis=1)
data_on_train_bottom = data_on_train_bottom.drop('train',axis=1)
data_on_train_both = data_on_train_both.drop('train',axis=1)
data_on_no_train = data_on_no_train.drop('train',axis=1)